In [1]:
from itertools import product
import numpy as np
import networkx as nx
import time

from adapt import adapt
from callbacks import MaxGradientStopper, ParameterStopper
from simulator import make_connectivity_pool
from scipy.optimize import minimize

In [13]:
num_qubits = 6
ham = np.random.uniform(size=(2**num_qubits, 2**num_qubits))
ham = ham + ham.T
ham /= 2.0

In [14]:
#pool = make_connectivity_pool(nx.path_graph(num_qubits))

In [15]:
pool = list(product(range(4), repeat=num_qubits))

In [16]:
def optimizer(*args, **kwargs):
    return minimize(*args, method='Nelder-Mead', **kwargs)

In [19]:
callbacks = [
    MaxGradientStopper(1e-5),
    ParameterStopper(50)
]

In [ ]:
t_i = time.time()

result = adapt(
    ham=ham,
    pool=pool,
    optimizer=optimizer,
    callbacks=callbacks
)

t_f = time.time()

print(f'Time elapsed: {np.floor(t_f-t_i)}s')

In [ ]:
result.step_history[-1]['energy'] - np.min(np.linalg.eigvalsh(ham))